In [6]:
import torch
import numpy as np
from torch.utils.data import TensorDataset
import pandas as pd
import json

In [7]:
test = pd.read_csv("dataset/test_preprocessed.csv")
train = pd.read_csv("dataset/train_preprocessed.csv")

In [8]:
# dropColumns = [
#     # "isFemale",
# 	# "isAmericanIndian",
# 	# "isAsian",
# 	# "isBlack",
# 	# "isPacificIslander",
# 	# "isWhite",
# 	# "isOtherRace",
# 	# "isHispanic",
#  	# "sameSexMarriage",
# 	# "mixedRaceMarriage",
# 	# "isMarried",
# 	# "wasMarried",
# 	# "neverMarried",
# 	# "bornInUS",
#     "index",
#     "SERIAL",
#     "PERNUM",
#     "HHWT",
#     "CLUSTER",
#     "STRATA",
#     "PERWT",
#     "RACNUM",
#     "YRMARR",
# 	"YRNATUR",
# ]

# test = test.drop(columns=dropColumns)
# train = train.drop(columns=dropColumns)

finds = [
	# "AGE",
	# "hasDegree",
	# "worksIn",
	# "occupation",
	"isFemale",
	"isAmericanIndian",
	"isAsian",
	"isBlack",
	"isPacificIslander",
	"isWhite",
	"isOtherRace",
	"isHispanic",
 	"sameSexMarriage",
	"mixedRaceMarriage",
	"isMarried",
	"wasMarried",
	"neverMarried",
	"bornInUS",
	"INCWAGE_CPIU_2010"
]


for column in test.columns:
	found = False
	for find in finds:
		if(column.find(find) != -1):
			found = True
			break
	if(not found):
		test = test.drop(columns=column)
		train = train.drop(columns=column)


for column in test.columns:
	test[column].fillna(False, inplace=True)
 
for column in train.columns:
	train[column].fillna(False, inplace=True)


In [11]:
print(test.columns)
print(train.columns)
print(train.shape)
print(test.shape)

Index(['INCWAGE_CPIU_2010', 'isFemale', 'isAmericanIndian', 'isAsian',
       'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace', 'isHispanic',
       'sameSexMarriage', 'mixedRaceMarriage', 'bornInUS', 'isMarried',
       'wasMarried', 'neverMarried'],
      dtype='object')
Index(['INCWAGE_CPIU_2010', 'isFemale', 'isAmericanIndian', 'isAsian',
       'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace', 'isHispanic',
       'sameSexMarriage', 'mixedRaceMarriage', 'bornInUS', 'isMarried',
       'wasMarried', 'neverMarried'],
      dtype='object')
(1231497, 15)
(217324, 15)


In [ ]:
#Remove all rows with INCWAGE_CPIU_2010 as 0
# train = train[train.INCWAGE_CPIU_2010 != 0]
# test = test[test.INCWAGE_CPIU_2010 != 0]

In [ ]:
# train['HOURS'] = train['WKSWORK1'] * train['UHRSWORK']
# train = train.drop(columns=['WKSWORK1', 'UHRSWORK'])

# test['HOURS'] = test['WKSWORK1'] * test['UHRSWORK']
# test = test.drop(columns=['WKSWORK1', 'UHRSWORK'])

In [ ]:
for column in test.columns:
	test[column] = test[column].astype(np.float32)

for column in train.columns:
	train[column] = train[column].astype(np.float32)


with open("dataset/columns.json", "w") as f:
	json.dump((test.drop(columns=['INCWAGE_CPIU_2010'])).columns.tolist(), f)

In [ ]:
salaryDivisor = 1
salaryCutoff = 100000
finalShift = 0

In [ ]:
xTrain = train.drop(columns=['INCWAGE_CPIU_2010'])
yTrain = train['INCWAGE_CPIU_2010']
xTest = test.drop(columns=['INCWAGE_CPIU_2010'])
yTest = test['INCWAGE_CPIU_2010']

In [ ]:
yTrain = yTrain.clip(upper=salaryCutoff)
yTest = yTest.clip(upper=salaryCutoff)

In [ ]:
yTrain = yTrain / salaryDivisor
yTest = yTest / salaryDivisor

In [ ]:
yTrain = yTrain - finalShift
yTest = yTest - finalShift

In [ ]:
AGE_mean = xTrain['AGE'].mean()
WKSWORK1_mean = xTrain['WKSWORK1'].mean()
UHRSWORK_mean = xTrain['UHRSWORK'].mean()
TRANTIME_mean = xTrain['TRANTIME'].mean()
# HOURS_mean = xTrain['HOURS'].mean()

xTrain['AGE'] = xTrain['AGE'] - AGE_mean
xTest['AGE'] = xTest['AGE'] - AGE_mean

xTrain['WKSWORK1'] = xTrain['WKSWORK1'] - WKSWORK1_mean
xTest['WKSWORK1'] = xTest['WKSWORK1'] - WKSWORK1_mean

xTrain['UHRSWORK'] = xTrain['UHRSWORK'] - UHRSWORK_mean
xTest['UHRSWORK'] = xTest['UHRSWORK'] - UHRSWORK_mean

xTrain['TRANTIME'] = xTrain['TRANTIME'] - TRANTIME_mean
xTest['TRANTIME'] = xTest['TRANTIME'] - TRANTIME_mean

# xTrain['HOURS'] = xTrain['HOURS'] - HOURS_mean
# xTest['HOURS'] = xTest['HOURS'] - HOURS_mean


In [ ]:
AGE_std = xTrain['AGE'].std()
WKSWORK1_std = xTrain['WKSWORK1'].std()
UHRSWORK_std = xTrain['UHRSWORK'].std()
TRANTIME_std = xTrain['TRANTIME'].std()
# HOURS_std = xTrain['HOURS'].std()


xTrain['AGE'] = xTrain['AGE'] / AGE_std
xTest['AGE'] = xTest['AGE'] / AGE_std

xTrain['WKSWORK1'] = xTrain['WKSWORK1'] / WKSWORK1_std
xTest['WKSWORK1'] = xTest['WKSWORK1'] / WKSWORK1_std

xTrain['UHRSWORK'] = xTrain['UHRSWORK'] / UHRSWORK_std
xTest['UHRSWORK'] = xTest['UHRSWORK'] / UHRSWORK_std

xTrain['TRANTIME'] = xTrain['TRANTIME'] / TRANTIME_std
xTest['TRANTIME'] = xTest['TRANTIME'] / TRANTIME_std

# xTrain['HOURS'] = xTrain['HOURS'] / HOURS_std
# xTest['HOURS'] = xTest['HOURS'] / HOURS_std


In [ ]:
# xTrain['AGE'] = xTrain['AGE'] / 100
# xTest['AGE'] = xTest['AGE'] / 100

# xTrain['WKSWORK1'] = xTrain['WKSWORK1'] / 52
# xTest['WKSWORK1'] = xTest['WKSWORK1'] / 52

# xTrain['UHRSWORK'] = xTrain['UHRSWORK'] / 168
# xTest['UHRSWORK'] = xTest['UHRSWORK'] / 168

# xTrain['TRANTIME'] = xTrain['TRANTIME'] / 163
# xTest['TRANTIME'] = xTest['TRANTIME'] / 163

In [ ]:
print(yTrain)
print(yTest)

In [ ]:
xTrainTensor = torch.tensor(xTrain.values, dtype=torch.float32)
yTrainTensor = torch.tensor(yTrain.values, dtype=torch.float32)

xTestTensor = torch.tensor(xTest.values, dtype=torch.float32)
yTestTensor = torch.tensor(yTest.values, dtype=torch.float32)

In [ ]:
print(xTrainTensor.shape)
print(yTrainTensor.shape)
print(xTestTensor.shape)
print(yTestTensor.shape)

torch.save(xTrainTensor, 'dataset/trainSetXNotProtected.pt')
torch.save(yTrainTensor, 'dataset/trainSetYNotProtected.pt')

torch.save(xTestTensor, 'dataset/testSetXNotProtected.pt')
torch.save(yTestTensor, 'dataset/testSetYNotProtected.pt')

In [ ]:
ax1 = yTrain.plot.hist(bins=10, alpha=0.5)